In [ ]:
import numpy as np
import pandas as pd
import gdspy
import matplotlib.pyplot as plt
from cytoolz import dissoc

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenbake as s
import fov
import fluid
from geometry import Cell, Round

In [ ]:
import pint

u = pint._DEFAULT_REGISTRY

In [ ]:
camera_dims = {
    "iris": (4.25, np.array([5056, 2960])),
    "bsi": (6.5, np.array([2048, 2048])),
}
# mags = (20, 40, 60, 100)
mags = (20, 40)
# mags = (40,)
fov_dims = {}
for camera, (pixel_size, fov_dim) in camera_dims.items():
    for mag in mags:
        fov_dims[f"{camera} {mag}x"] = fov_dim * pixel_size / mag

In [ ]:
def print_flow_info(height, Q_per_snake, mu, nu, metadata):
    for name, md in metadata.items():
        R_, extra = fluid.snake_resistance(
            height, md, length_unit=u.um, return_extra=True
        )
        R = R_ * mu
        # print('R', R.to(u.mbar * u.min / u.uL))
        delta_P = R * Q_per_snake
        print(f"{name}: {delta_P.to(u.bar)}")
        if "manifold_split" in md:
            flow_rate = Q_per_snake * md["manifold_split"]
        else:
            flow_rate = Q_per_snake
        print(f"    flow rate: {flow_rate}")
        if "flow_nonuniformity" in extra:
            print(f"    flow nonuniformity: {extra['flow_nonuniformity']}")
        if "manifold_width" in md:
            L_e = fluid.entrance_length(
                height,
                md["manifold_width"] * u.um,
                Q_per_snake * md["manifold_split"],
                nu,
            )
            print(f"    entrance length: {L_e}")

# Designs

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims = np.array([33e3, 18e3])
chip_dims0 = np.array([23e3, 13e3])
chip_dims1 = np.array([40e3, 22e3])

In [ ]:
plt.figure(figsize=(10, 5))
ax = plt.gca()
ax.add_patch(plt.Rectangle(-coverslip_dims / 2, *coverslip_dims, fill=False))
ax.add_patch(plt.Rectangle(-chip_dims0 / 2, *chip_dims0, fill=False, ls="-."))
# ax.add_patch(plt.Rectangle(-chip_dims1 / 2, *chip_dims1, fill=False, ls="--"))
ax.add_patch(plt.Rectangle(-chip_dims / 2, *chip_dims, fill=False))
ax.set_xlim(-1.2 * coverslip_dims[0] / 2, 1.2 * coverslip_dims[0] / 2)
ax.set_ylim(-1.2 * coverslip_dims[1] / 2, 1.2 * coverslip_dims[1] / 2)
ax.set_aspect("equal")

In [ ]:
%%time
metadata = {}
base_params = dict(
    design_func=s.snake,
    dims=chip_dims,
    split=8,
    gap_lanes=0,
    trench_length=35,
    feeding_channel_width=80,
    #     horizontal_margin=2e3,
    trench_gap=20,
    trench_width=1.4,
    # trench_spacing=2.1,
    border_margin=0.5e3,
    port_margin=1e3,  # 0.6
    port_radius=750 / 2,
    # port_radius=0,
    port=False,
    registration_marks=True,
    registration_mark_barcodes=False,
    #     chip_id=0,
    #     mark_size=2,
    #     mark_spacing=1.5,
    draw_trenches=True,
    metadata=metadata,
)
manifold_params = dissoc(
    {
        **base_params,
        **dict(
            design_func=s.manifold_snake,
            manifold_input_style="bend-in",
            dims=chip_dims,
            split=None,
            num_manifolds=1,
            manifold_width=200,
            manifold_input_margin=2e3,
            manifold_bend_margin=0.2e3,
            manifold_bend_radius=300,
            manifold_margin=100,
            # feeding_channel_width=40,
            # port_wayfinder_orientations=("right", "top")#("right", "top", "bottom"),
            #             port_margin=0.5e3,
            # trench_spacing=2.1,
        ),
    },
    "horizontal_margin",
    "gap_lanes",
)

# dense_chip = dict(
#     manifold_input_style="u-turn",
#     feeding_channel_width=30,
#     trench_length=30,
#     trench_gap=20,
#     lanes_per_snake=3,
#     trench_spacing=2.1,
# )
dense_chip = dict(
    feeding_channel_width=40,
    trench_gap=20,
    lanes_per_snake=3,
    trench_spacing=2.1,
)
# dense_chip2 = dict(
#     feeding_channel_width=35,
#     trench_gap=20,
#     lanes_per_snake=3,
#     trench_spacing=2.1,
# )
workhorse = dict(
    feeding_channel_width=90,
    trench_gap=30,
    lanes_per_snake=5,
    trench_spacing=2.1,
)
workhorse_poormedia = dict(
    feeding_channel_width=90,
    trench_width=1.2,
    trench_gap=30,
    lanes_per_snake=5,
    trench_spacing=2.1,
)

params = [dense_chip, workhorse, workhorse, workhorse, workhorse_poormedia, dense_chip]
params = [{**manifold_params, **p} for p in params]
# chip names
chip_names = [
    "Basilisk {ports} FC{feeding_channel_width} L{trench_length} W{trench_width} TS{trench_spacing}{reg} TG{trench_gap}".format(
        ports=f"S{p['split']}"
        if np.isscalar(p["split"])
        else "S{} LS{}".format(p["num_manifolds"], p["lanes_per_snake"]),
        reg=f" M{p.get('mark_size')} ID{p.get('chip_id')}"
        if p.get("registration_mark_barcodes")
        else "",
        **p,
    )
    for p in params
]
# make chip cells
chips = [s.chip(name, **p) for p, name in zip(params, chip_names)]
# calculate FOV info and overlay FOVs on chip cells
grid_metadata = {}
draw_grid = False
for chip_name, chip in zip(chip_names, chips):
    chip_grid_metadata = fov.get_grid_metadata(fov_dims, metadata[chip_name])
    grid_metadata[chip_name] = chip_grid_metadata
    if draw_grid:
        fov.draw_grid_overlay(
            chip,
            metadata[chip_name],
            fov_dims,
            chip_grid_metadata,
            center_margins=True,
            rotate=True,
        )
grid_df = pd.concat(
    {
        k: pd.DataFrame(v.values()).set_index("fov_name")
        for k, v in grid_metadata.items()
    },
    names=["chip"],
)
# lay out wafer, output design
wafer_manifest = "\n".join([f"{idx+1}) " + name for idx, name in enumerate(chip_names)])
main_cell = s.wafer(
    chips,
    text_right="Basilisk\n JQS/DE/YG 210307",
    text_left=wafer_manifest,
    text=True,
    mask=False,
    chip_dims=chip_dims,
    diameter=(4 * u.inch).to(u.um).magnitude,
    alignment_mark_position=40e3,
)
s.write_gds(main_cell, "designs/210307basilisk.gds")

In [ ]:
print(wafer_manifest)

In [ ]:
{k: v["num_trenches"] for k, v in metadata.items()}

In [ ]:
{k: v["num_lanes"] for k, v in metadata.items()}

In [ ]:
Q_per_snake = 20 * u.uL / u.min
height = 75 * u.um
mu = 0.7 * u.cP  # dynamic viscosity
nu = mu / (1 * u.g / u.cm**3)  # kinematic viscosity
print_flow_info(height, Q_per_snake, mu, nu, metadata)

In [ ]:
grid_df.loc[pd.IndexSlice[:, ["iris 40x", "bsi 40x"]], :].round(2)

In [ ]:
grid_df.loc[pd.IndexSlice[:, ["iris 20x", "bsi 20x"]], :].round(2)

# Old chip calculations

In [ ]:
(
    Q_per_snake
    * mu
    * fluid.snake_resistance(
        80 * u.um,
        {"snake_length": [17920 * 19], "feeding_channel_width": 90},
        length_unit=u.um,
    )
).to(u.bar)

In [ ]:
(
    Q_per_snake
    * mu
    * fluid.snake_resistance(
        80 * u.um,
        {"snake_length": [17920 * 25], "feeding_channel_width": 45},
        length_unit=u.um,
    )
).to(u.bar)

# FOV calculation redux

In [ ]:
md = metadata["Basilisk S1 LS5 FC90 L35 W1.4 TS5.1 TG30"]

In [ ]:
a = fov.get_fov_coverage(fov_dims["bsi 40x"], **md, skip_first=False)
b = fov.get_fov_coverage(fov_dims["bsi 40x"], **md, skip_first=True)
(a, b)

In [ ]:
fovh = fov_dims["bsi 40x"][0]

In [ ]:
for k, v in md.items():
    locals()[k] = v

In [ ]:
delta_y1 = trench_gap + trench_length
delta_y2 = feeding_channel_width + trench_length
delta_y = delta_y1 + delta_y2

In [ ]:
(delta_y1, delta_y2, delta_y)

In [ ]:
np.array((delta_y1, delta_y2, delta_y)) * 2

In [ ]:
divmod(fovh - trench_length, delta_y)

In [ ]:
unit_cell = trench_gap / 2 + trench_length + feeding_channel_width / 2

In [ ]:
unit_cell

In [ ]:
num_unit_cells, remainder = divmod(fovh, unit_cell)
(num_unit_cells, remainder, num_unit_cells * unit_cell)

In [ ]:
partial_delta_y1 = trench_length + trench_gap / 2
partial_delta_y2 = trench_length + feeding_channel_width / 2

In [ ]:
(partial_delta_y1, partial_delta_y2)

In [ ]:
if num_unit_cells % 2 == 0:
    print(divmod(remainder, partial_delta_y1), divmod(remainder, partial_delta_y2))

In [ ]:
print(divmod(remainder, partial_delta_y1), divmod(remainder, partial_delta_y2))